In [1]:
"""
PASO 1: Captura de los 8 gestos
100 muestras por gesto
"""

import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
from collections import deque

# ========== CONFIGURACIÓN ==========
SEQUENCE_LENGTH = 15
DATA_DIR = "../gesture_data"
SAMPLES_PER_GESTURE = 100

os.makedirs(DATA_DIR, exist_ok=True)

# ========== TODOS LOS GESTOS ==========
ALL_GESTURES = [
    "SWIPE_LEFT",
    "SWIPE_RIGHT",
    "SWIPE_UP",
    "SWIPE_DOWN",
    "PINCH_OPEN",
    "PINCH_CLOSE",
    "FIST_CLOSE",
    "OPEN_STATIC"
]

# ========== INSTRUCCIONES ==========
INSTRUCTIONS = {
    "SWIPE_LEFT": [
        "Desliza la mano hacia la IZQUIERDA",
        "Movimiento horizontal claro"
    ],
    "SWIPE_RIGHT": [
        "Desliza la mano hacia la DERECHA",
        "Movimiento horizontal claro"
    ],
    "SWIPE_UP": [
        "Desliza la mano hacia ARRIBA",
        "Movimiento vertical claro"
    ],
    "SWIPE_DOWN": [
        "Desliza la mano hacia ABAJO",
        "Movimiento vertical claro"
    ],
    "PINCH_OPEN": [
        "Empieza: PELLIZCO (indice-pulgar)",
        "Termina: MANO ABIERTA (5 dedos)"
    ],
    "PINCH_CLOSE": [
        "Empieza: MANO ABIERTA (5 dedos)",
        "Termina: PELLIZCO (indice-pulgar)"
    ],
    "FIST_CLOSE": [
        "Empieza: MANO ABIERTA",
        "CIERRA el PUÑO (rapido y claro)"
    ],
    "OPEN_STATIC": [
        "MANO ABIERTA con 5 dedos",
        "QUIETA - No muevas la mano"
    ]
}

# ========== MEDIAPIPE ==========
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def capture_gesture(gesture_name, num_samples):
    """Capturar muestras de un gesto"""
    
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    )
    
    cap = cv2.VideoCapture(0)
    samples = []
    sequence_buffer = deque(maxlen=SEQUENCE_LENGTH)
    recording = False
    
    print(f"\n{'='*70}")
    print(f"CAPTURANDO: {gesture_name}")
    print(f"{'='*70}")
    print(f"Objetivo: {num_samples} muestras")
    print(f"\nINSTRUCCIONES:")
    for i, inst in enumerate(INSTRUCTIONS[gesture_name], 1):
        print(f"  {i}. {inst}")
    print(f"\n{'='*70}")
    print("ESPACIO = Grabar | Q = Siguiente gesto")
    print(f"{'='*70}\n")
    
    while len(samples) < num_samples:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        height, width = frame.shape[:2]
        
        # Panel superior negro
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (width, 220), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
        
        # Título
        cv2.putText(frame, f"GESTO: {gesture_name}", (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
        
        # Instrucciones
        y = 80
        for inst in INSTRUCTIONS[gesture_name]:
            cv2.putText(frame, inst, (20, y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            y += 30
        
        # Progreso
        percentage = (len(samples) / num_samples) * 100
        cv2.putText(frame, f"Progreso: {len(samples)}/{num_samples} ({percentage:.0f}%)",
                   (20, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Estado
        if recording:
            cv2.putText(frame, "GRABANDO...", (20, 210),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            cv2.circle(frame, (width - 50, 50), 20, (0, 0, 255), -1)
        else:
            cv2.putText(frame, "Presiona ESPACIO", (20, 210),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (200, 200, 200), 2)
        
        # Landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )
                
                if recording:
                    landmarks = hand_landmarks.landmark
                    landmark_array = np.array([[lm.x, lm.y, lm.z] for lm in landmarks])
                    sequence_buffer.append(landmark_array)
                    
                    # Barra de buffer
                    buffer_progress = len(sequence_buffer)
                    bar_x = width - 420
                    bar_y = height - 50
                    bar_width = 400
                    
                    cv2.rectangle(frame, (bar_x, bar_y), 
                                 (bar_x + bar_width, bar_y + 20), (50, 50, 50), -1)
                    
                    fill = int((buffer_progress / SEQUENCE_LENGTH) * bar_width)
                    cv2.rectangle(frame, (bar_x, bar_y),
                                 (bar_x + fill, bar_y + 20), (0, 255, 255), -1)
                    
                    cv2.putText(frame, f"Buffer: {buffer_progress}/{SEQUENCE_LENGTH}",
                               (bar_x, bar_y - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                    
                    if len(sequence_buffer) == SEQUENCE_LENGTH:
                        samples.append(list(sequence_buffer))
                        print(f"✅ Muestra {len(samples)}/{num_samples} ({percentage:.0f}%)")
                        sequence_buffer.clear()
                        recording = False
        else:
            cv2.putText(frame, "NO HAND DETECTED", (20, height - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('Captura de Gestos', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):
            if not recording and results.multi_hand_landmarks:
                recording = True
                sequence_buffer.clear()
                print(f"🔴 Grabando muestra {len(samples)+1}...")
        elif key == ord('q') or key == ord('Q'):
            break
    
    cap.release()
    hands.close()
    
    # Guardar
    if samples:
        file_path = os.path.join(DATA_DIR, f"{gesture_name}.pkl")
        with open(file_path, 'wb') as f:
            pickle.dump(samples, f)
        print(f"\n✅ {len(samples)} muestras guardadas: {file_path}\n")
    
    return len(samples)

# ========== MAIN ==========
def main():
    print("\n" + "="*70)
    print("CAPTURA DE GESTOS - Sistema Completo")
    print("="*70)
    print(f"\nGestos: {len(ALL_GESTURES)}")
    print(f"Muestras por gesto: {SAMPLES_PER_GESTURE}")
    print(f"Total: {len(ALL_GESTURES) * SAMPLES_PER_GESTURE} muestras\n")
    
    for i, g in enumerate(ALL_GESTURES, 1):
        print(f"{i}. {g}")
    
    print("\n" + "="*70)
    input("Presiona ENTER para comenzar...")
    
    results = {}
    
    for i, gesture in enumerate(ALL_GESTURES, 1):
        print(f"\n{'='*70}")
        print(f"GESTO {i}/{len(ALL_GESTURES)}")
        print(f"{'='*70}")
        
        count = capture_gesture(gesture, SAMPLES_PER_GESTURE)
        results[gesture] = count
        
        if i < len(ALL_GESTURES):
            print(f"\n{'='*70}")
            print(f"Siguiente: {ALL_GESTURES[i]}")
            print("ENTER = Continuar | Q = Salir")
            print(f"{'='*70}\n")
            
            if input().lower() == 'q':
                break
    
    cv2.destroyAllWindows()
    
    # Resumen
    print("\n" + "="*70)
    print("RESUMEN")
    print("="*70)
    total = 0
    for gesture, count in results.items():
        print(f"{gesture}: {count} muestras")
        total += count
    print(f"\nTOTAL: {total} muestras")
    print("="*70)
    print("\n✅ Captura completada!")
    print("Siguiente: Ejecutar 2_train_model.ipynb")

if __name__ == "__main__":
    main()


CAPTURA DE GESTOS - Sistema Completo

Gestos: 8
Muestras por gesto: 100
Total: 800 muestras

1. SWIPE_LEFT
2. SWIPE_RIGHT
3. SWIPE_UP
4. SWIPE_DOWN
5. PINCH_OPEN
6. PINCH_CLOSE
7. FIST_CLOSE
8. OPEN_STATIC


GESTO 1/8

CAPTURANDO: SWIPE_LEFT
Objetivo: 100 muestras

INSTRUCCIONES:
  1. Desliza la mano hacia la IZQUIERDA
  2. Movimiento horizontal claro

ESPACIO = Grabar | Q = Siguiente gesto



c:\ELOI\PROGRAMACION\PROYECTOS\LAUZHACK\LAUZHACK\app\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


🔴 Grabando muestra 1...
✅ Muestra 1/100 (0%)
🔴 Grabando muestra 2...
✅ Muestra 2/100 (1%)
🔴 Grabando muestra 3...
✅ Muestra 3/100 (2%)
🔴 Grabando muestra 4...
✅ Muestra 4/100 (3%)
🔴 Grabando muestra 5...
✅ Muestra 5/100 (4%)
🔴 Grabando muestra 6...
✅ Muestra 6/100 (5%)
🔴 Grabando muestra 7...
✅ Muestra 7/100 (6%)
🔴 Grabando muestra 8...
✅ Muestra 8/100 (7%)
🔴 Grabando muestra 9...
✅ Muestra 9/100 (8%)
🔴 Grabando muestra 10...
✅ Muestra 10/100 (9%)
🔴 Grabando muestra 11...
✅ Muestra 11/100 (10%)
🔴 Grabando muestra 12...
✅ Muestra 12/100 (11%)
🔴 Grabando muestra 13...
✅ Muestra 13/100 (12%)
🔴 Grabando muestra 14...
✅ Muestra 14/100 (13%)
🔴 Grabando muestra 15...
✅ Muestra 15/100 (14%)
🔴 Grabando muestra 16...
✅ Muestra 16/100 (15%)
🔴 Grabando muestra 17...
✅ Muestra 17/100 (16%)
🔴 Grabando muestra 18...
✅ Muestra 18/100 (17%)
🔴 Grabando muestra 19...
✅ Muestra 19/100 (18%)
🔴 Grabando muestra 20...
✅ Muestra 20/100 (19%)
🔴 Grabando muestra 21...
✅ Muestra 21/100 (20%)
🔴 Grabando muestra 2